### Objective

### Installation and Setup

### Twitter API

# Lets create twitter object and use its API. Code snippets below will show you how to use this API

In [1]:
from TwitterAPI import TwitterAPI, TwitterOAuth


In [2]:
o = TwitterOAuth.read_file(r'C:\Users\Adarsh\Desktop\credentials.txt')

In [3]:
o.access_token_key

'1389546229-2ZPMSkYlxLue2sWPtX2km4izxJSRKfWRtnoHibo'

In [4]:
# Using OAuth1...
twitter = TwitterAPI(o.consumer_key,
                 o.consumer_secret,
                 o.access_token_key,
                 o.access_token_secret)

In [ ]:
help(twitter)

In [ ]:
# What can we do with this twitter object?
# builtin method `dir` tells us...
dir(twitter)

In [ ]:
twitter.auth

In [ ]:
# Get help on the `request` method using the builtin method called...`help`
help(twitter.request)

In [ ]:
# Let's start by querying the search API
response = twitter.request('search/tweets', {'q': 'big+data'}) 

In [ ]:
# What object is returned?
# builtin type method will tell us.
print(type(response))
dir(response)

In [ ]:
response.json

In [ ]:
response.status_code
# See https://dev.twitter.com/overview/api/response-codes

In [ ]:
tweets = [r for r in response]

In [ ]:
print('found %d tweets' % len(tweets))

In [ ]:
type(tweets)

In [ ]:
type(tweets[0])

In [ ]:
tweets[0]

In [ ]:
help(tweets[0])

In [ ]:
tweets[0].keys()

In [ ]:
tweets[0]['text']

In [ ]:
tweets[0]['created_at']

In [ ]:
tweets[0]['text']

In [ ]:
tweets[0]['user']

In [ ]:
user = tweets[0]['user']
print('screen_name=%s, name=%s, location=%s' % (user['screen_name'], user['name'], user['location']))

In [ ]:
# Who follows this person?
# https://dev.twitter.com/docs/api/1.1/get/followers/list
screen_name = user['screen_name']
response  = twitter.request('followers/list', {'screen_name': screen_name, 'count':200})
followers = [follower for follower in response]
        
print('found %d followers for %s' % (len(followers), screen_name))
# See more about paging here: https://dev.twitter.com/docs/working-with-timelines

In [ ]:
print(followers[0]['screen_name'])

##  Limitations:       Can only search 2 weeks in past         But can get up to 3,200 most recent tweets of a user     Rate limits! https://dev.twitter.com/docs/rate-limiting/1.1/limits         e.g., 180 requests in 15 minute window 

# Get BestBuy timeline for the deals screen-name

# This is the screen name for BestBuy_Deals

In [5]:
# Get BestBuyDeals timeline = ''
screen_name = 'BestBuy_Deals'
timeline = [tweet for tweet in twitter.request('statuses/user_timeline',
                                                {'screen_name': screen_name,
                                                 'count': 200})]
print('got %d tweets for user %s' % (len(timeline), screen_name))

got 200 tweets for user BestBuy_Deals


In [6]:
# Print time got created.

timeline[3]['created_at']

'Wed Aug 28 13:00:00 +0000 2019'

In [7]:
# Print the text.
print('\n\n\n'.join(t['text'] for t in timeline))

Save $150 on the Denon 2.1 Channel Soundbar System with Wireless Subwoofer - Black. #Deal


Save $130 on the Lenovo 330S 15IKB 15.6" Laptop Intel Core i3 4GB Memory 128GB Solid State Drive - Midnight Blue. #Deal


Save $100 on the Bang &amp; Olufsen Beoplay H8i Wireless Noise Canceling On-Ear Headphones - Black. #Deal


Save $75 on the OtterBox Trooper 20 Soft Cooler Hazy Harbor. #Deal


Save $130 on the Sharp 50" Class LED 1080p Smart HDTV Roku TV. #Deal


Save $100 on the Toshiba 49" Class LED 1080p Smart HDTV Fire TV Edition. #Deal


Save $100 on the Insignia 50" Class LED 1080p HDTV. #Deal


Save $150 on the HP 14" Touch Screen Laptop Intel Core i3 4GB Memory 128GB Solid State Drive - Ash Silver Keyboard Frame. #Deal


Save $250 on the Lenovo Yoga 730 2 in 1 15.6" Touch Screen Laptop Intel Core i5 8GB Memory 256GB Solid State Drive - Iron Gray. #Deal


Save $250 on the Epson Refurbished Pro EX9220 1080p Wireless 3LCD Projector - Black. #Deal


RT @BestBuy: The new Samsung Galaxy No

In [8]:
# Count words
from collections import Counter  # This is just a fancy dict mapping from object->int, starting at 0.
counts = Counter()
for tweet in timeline:
    counts.update(tweet['text'].lower().split())
print('found %d unique terms in %d tweets' % (len(counts), len(timeline)))
counts.most_common(10)

found 787 unique terms in 200 tweets


[('the', 201),
 ('on', 192),
 ('save', 189),
 ('#deal', 179),
 ('-', 127),
 ('and', 51),
 ('led', 49),
 ('black.', 47),
 ('class', 46),
 ('smart', 42)]

In [9]:
import re
for tweet in timeline:
    deal = tweet['text']
    print(deal + '\n')


Save $150 on the Denon 2.1 Channel Soundbar System with Wireless Subwoofer - Black. #Deal

Save $130 on the Lenovo 330S 15IKB 15.6" Laptop Intel Core i3 4GB Memory 128GB Solid State Drive - Midnight Blue. #Deal

Save $100 on the Bang &amp; Olufsen Beoplay H8i Wireless Noise Canceling On-Ear Headphones - Black. #Deal

Save $75 on the OtterBox Trooper 20 Soft Cooler Hazy Harbor. #Deal

Save $130 on the Sharp 50" Class LED 1080p Smart HDTV Roku TV. #Deal

Save $100 on the Toshiba 49" Class LED 1080p Smart HDTV Fire TV Edition. #Deal

Save $100 on the Insignia 50" Class LED 1080p HDTV. #Deal

Save $150 on the HP 14" Touch Screen Laptop Intel Core i3 4GB Memory 128GB Solid State Drive - Ash Silver Keyboard Frame. #Deal

Save $250 on the Lenovo Yoga 730 2 in 1 15.6" Touch Screen Laptop Intel Core i5 8GB Memory 256GB Solid State Drive - Iron Gray. #Deal

Save $250 on the Epson Refurbished Pro EX9220 1080p Wireless 3LCD Projector - Black. #Deal

RT @BestBuy: The new Samsung Galaxy Note10 and N

# Find  the deals in the BestBuy_Deal tweets  that match products in SmartPortables MySQL product table

In [12]:
import re
import pymysql

cnx = pymysql.connect(user='root', password='root',
                              host='127.0.0.1',
                              database='bestdealdatabase')
cursor = cnx.cursor()

query = ("SELECT id FROM ProductDetails")
cursor.execute(query)


dealMatchGauranteed=[]
for product in cursor:
    for tweet in timeline:
        deal = (tweet['text'])
        if (len(re.findall('\s'+product[0]+'\s',deal)) >= 1):
            dealMatchGauranteed = dealMatchGauranteed + [deal]
        

In [13]:
# Sanity Test that  we got some deals
dealMatchGauranteed[:5]

['Save $150 on the Sanus Premium Series Full Motion TV Wall Mount for Most 42" 90" Flat Panel TVs - Black. #Deal',
 'Save $50 on the Insignia Premium Audio Pico WVGA DLP Projector - Black. #Deal']

# Create and write the deals into DealMatches.txt file that will be used by web-app of SmartPortables to display two deal matches

In [15]:
dealMatchFile = open(r'C:\Users\Adarsh\Desktop\DealMatches.txt', 'w')
for deal in dealMatchGauranteed:
    dealMatchFile.write("%s\n" % deal)

dealMatchFile.close()